In [1]:
import pandas as pd
from pandas import np
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:

m2 = pd.read_csv("/content/gdrive/My Drive/finalmerge.csv",index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
m2

,Unnamed: 0_x,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin,Dest,Unnamed: 0_y,airport_x,date_x,windspeedKmph_x,winddirDegree_x,weatherCode_x,precipMM_x,visibility_x,pressure_x,cloudcover_x,DewPointF_x,WindGustKmph_x,tempF_x,WindChillF_x,humidity_x,time_x,Unnamed: 0.1,airport_y,date_y,windspeedKmph_y,winddirDegree_y,weatherCode_y,precipMM_y,visibility_y,pressure_y,cloudcover_y,DewPointF_y,WindGustKmph_y,tempF_y,WindChillF_y,humidity_y,time_y
0,57,2016-01-01,1,2016,1,1,741.0,0.0,700,0.0,14747,12478,1610.0,1600,0.0,8.0,SEA,JFK,16,JFK,2016-01-01,22,255,113,0.0,10,1015,0,32,26,41,32,74,1600,7,SEA,2016-01-01,7,107,113,0.0,10,1030,0,23,8,34,29,66,700
1,219870,2016-01-01,1,2016,1,1,705.0,0.0,700,0.0,14747,12892,934.0,900,0.0,0.0,SEA,LAX,9,LAX,2016-01-01,12,31,113,0.0,10,1020,2,30,15,51,47,46,900,7,SEA,2016-01-01,7,107,113,0.0,10,1030,0,23,8,34,29,66,700
2,356730,2016-01-01,1,2016,1,1,658.0,0.0,700,3.0,14747,12892,933.0,900,0.0,0.0,SEA,LAX,9,LAX,2016-01-01,12,31,113,0.0,10,1020,2,30,15,51,47,46,900,7,SEA,2016-01-01,7,107,113,0.0,10,1030,0,23,8,34,29,66,700
3,334974,2016-01-01,1,2016,1,1,723.0,0.0,700,6.0,14747,12478,1534.0,1500,0.0,4.0,SEA,JFK,15,JFK,2016-01-01,21,254,113,0.0,10,1016,0,33,24,42,33,74,1500,7,SEA,2016-01-01,7,107,113,0.0,10,1030,0,23,8,34,29,66,700
4,356750,2016-01-01,1,2016,1,1,726.0,0.0,700,0.0,14747,12892,1022.0,1000,0.0,3.0,SEA,LAX,10,LAX,2016-01-01,10,39,113,0.0,10,1020,9,29,13,53,49,42,1000,7,SEA,2016-01-01,7,107,113,0.0,10,1030,0,23,8,34,29,66,700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815400,453453,2017-09-18,3,2017,9,18,2240.0,0.0,2200,0.0,12478,12892,134.0,200,0.0,0.0,JFK,LAX,410,LAX,2017-09-18,5,220,113,0.0,10,1012,1,70,6,74,74,87,200,430,JFK,2017-09-18,19,60,122,0.0,10,1018,98,67,26,69,69,92,2200
1815401,455805,2017-09-21,3,2017,9,21,2358.0,1.0,2200,73.0,12478,12892,247.0,200,1.0,51.0,JFK,LAX,482,LAX,2017-09-21,11,179,143,0.0,10,1010,56,63,12,65,65,91,200,502,JFK,2017-09-21,18,28,122,0.0,10,1015,96,64,26,72,72,75,2200
1815402,456636,2017-09-22,3,2017,9,22,2240.0,0.0,2200,0.0,12478,12892,141.0,200,0.0,0.0,JFK,LAX,506,LAX,2017-09-22,10,135,113,0.0,10,1010,3,59,12,67,67,75,200,526,JFK,2017-09-22,22,14,116,0.0,10,1014,48,57,33,71,71,63,2200
1815403,457242,2017-09-23,3,2017,9,23,2146.0,0.0,2100,3.0,12478,12892,21.0,100,0.0,0.0,JFK,LAX,529,LAX,2017-09-23,6,202,113,0.0,10,1009,4,61,8,68,68,80,100,549,JFK,2017-09-23,12,1,113,0.0,10,1017,0,68,15,86,86,55,2100


In [0]:
X=['windspeedKmph_x', 'winddirDegree_x', 'weatherCode_x', 'precipMM_x', 'visibility_x',
           'pressure_x', 'cloudcover_x', 'DewPointF_x', 'WindGustKmph_x', 'tempF_x',
           'WindChillF_x', 'humidity_x']
x = m2[X] #X value containing all data from feature columns
y = m2.ArrDel15

In [0]:
x = m2[X] #X value containing all data from feature columns
y = m2.ArrDel15 #y value containing target variable of arrival delay(0s and 1s)

In [0]:
#resampling
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
x = pd.concat([X_train, y_train], axis=1)

In [0]:
not_delayed = x[x.ArrDel15==0]
delayed = x[x.ArrDel15==1]

In [0]:
delayed_upsampled = resample(delayed,
                          replace=True, # sample with replacement
                          n_samples=len(not_delayed), # match number in majority class
                          random_state=27)
upsampled = pd.concat([not_delayed, delayed_upsampled])
x = upsampled[X]
y = upsampled.ArrDel15

In [30]:
#logistic regression model implementation
from sklearn.linear_model import LogisticRegression
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [32]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
pd.DataFrame(cnf_matrix)

,0,1
0,169867,99742
1,136871,131925


In [33]:
#logistic reg report for taking only the weather data at the origin airport as features
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.55      0.63      0.59    269609
         1.0       0.57      0.49      0.53    268796

    accuracy                           0.56    538405
   macro avg       0.56      0.56      0.56    538405
weighted avg       0.56      0.56      0.56    538405



In [34]:
report = classification_report(y_test, y_pred, output_dict=True)
df = pd.DataFrame(report).transpose()
df

,precision,recall,f1-score,support
0.0,0.553785,0.630049,0.589461,269609.00000
1.0,0.569460,0.490800,0.527212,268796.00000
accuracy,0.560530,0.560530,0.560530,0.56053
macro avg,0.561622,0.560425,0.558336,538405.00000
weighted avg,0.561611,0.560530,0.558383,538405.00000


In [0]:
#decisiontree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred =  clf.predict(X_test)

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.73      0.68      0.70    269609
         1.0       0.70      0.74      0.72    268796

    accuracy                           0.71    538405
   macro avg       0.71      0.71      0.71    538405
weighted avg       0.71      0.71      0.71    538405



In [39]:
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.ensemble import ExtraTreesClassifier
etc=ExtraTreesClassifier(n_jobs=-1,random_state = None)
etc.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [0]:
feature_importance = etc.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        etc.estimators_], 
                                        axis = 0) 

In [42]:
X_train.describe()

,windspeedKmph_x,winddirDegree_x,weatherCode_x,precipMM_x,visibility_x,pressure_x,cloudcover_x,DewPointF_x,WindGustKmph_x,tempF_x,WindChillF_x,humidity_x
count,1.615215e+06,1.615215e+06,1.615215e+06,1.615215e+06,1.615215e+06,1.615215e+06,1.615215e+06,1.615215e+06,1.615215e+06,1.615215e+06,1.615215e+06,1.615215e+06
mean,1.286952e+01,1.899360e+02,1.502325e+02,3.268177e-01,9.453160e+00,1.015686e+03,4.319735e+01,5.001704e+01,1.740371e+01,6.614798e+01,6.490739e+01,6.291240e+01
std,6.634096e+00,8.474405e+01,7.464407e+01,1.158920e+00,1.753712e+00,6.319580e+00,3.956732e+01,1.607434e+01,9.388684e+00,1.714828e+01,1.898855e+01,2.256815e+01
min,0.000000e+00,0.000000e+00,1.130000e+02,0.000000e+00,0.000000e+00,9.810000e+02,0.000000e+00,-1.500000e+01,1.000000e+00,-1.000000e+01,-2.500000e+01,3.000000e+00
25%,8.000000e+00,1.280000e+02,1.130000e+02,0.000000e+00,1.000000e+01,1.012000e+03,0.000000e+00,3.900000e+01,1.000000e+01,5.600000e+01,5.500000e+01,4.800000e+01
50%,1.200000e+01,1.990000e+02,1.160000e+02,0.000000e+00,1.000000e+01,1.016000e+03,3.600000e+01,5.200000e+01,1.600000e+01,6.700000e+01,6.700000e+01,6.700000e+01
75%,1.700000e+01,2.550000e+02,1.220000e+02,1.000000e-01,1.000000e+01,1.019000e+03,8.500000e+01,6.200000e+01,2.200000e+01,7.900000e+01,7.800000e+01,8.100000e+01
max,5.900000e+01,3.600000e+02,3.950000e+02,3.050000e+01,2.000000e+01,1.046000e+03,1.000000e+02,8.300000e+01,8.200000e+01,1.210000e+02,1.210000e+02,1.000000e+02


In [43]:
feature_importance

array([0.09001909, 0.1145931 , 0.05620445, 0.04205846, 0.02936791,
       0.09950967, 0.07922842, 0.10021664, 0.09657653, 0.09442762,
       0.09471834, 0.10307977])

In [45]:
y_pred = etc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.73      0.68      0.70    269609
         1.0       0.70      0.74      0.72    268796

    accuracy                           0.71    538405
   macro avg       0.71      0.71      0.71    538405
weighted avg       0.71      0.71      0.71    538405



In [0]:
#xgboost
from xgboost import XGBClassifier

In [47]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [48]:
y_pred = model.predict(X_test)
print(y_pred)

[0. 0. 0. ... 0. 0. 1.]


In [51]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.58      0.61      0.59    269609
         1.0       0.58      0.55      0.57    268796

    accuracy                           0.58    538405
   macro avg       0.58      0.58      0.58    538405
weighted avg       0.58      0.58      0.58    538405

